In [15]:
# install the libraries
from sklearn.preprocessing import MinMaxScaler
import os
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_forecasting import TimeSeriesDataSet, NBeats, Baseline
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import SMAPE

In [16]:
my_early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        patience=3,
        verbose=False,
        mode='min',
        strict=True
    )

In [17]:
file = "C:/Users/gurpr/Videos/BetaSci/Rossmann_Store1_Data_FULL.csv"
pwd = os.getcwd()
os.chdir(os.path.dirname(file))
data = pd.read_csv(os.path.basename(file), usecols = [1],
                      engine = "python",
                      skipfooter = 3)

data.head()

,Sale
0,4327
1,4486
2,4997
3,7176
4,5580


In [18]:
#Data Prep
# Get the raw data values from the pandas data frame.
data_raw = data.values.astype("float32")

# We apply the MinMax scaler from sklearn
# to normalize data in the (0, 1) interval.
scaler = MinMaxScaler(feature_range = (0, 1))
dataset = scaler.fit_transform(data_raw)

# Print a few values.
dataset[0:5]

array([[0.27421156],
       [0.29639968],
       [0.3677086 ],
       [0.67178345],
       [0.449065  ]], dtype=float32)

In [19]:
# Using 75% of data for training, 25% for validation.
TRAIN_SIZE = 0.75

train_size = int(len(dataset) * TRAIN_SIZE)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print("Number of entries (training set, test set): " + str((len(train), len(test))))

Number of entries (training set, test set): (582, 195)


In [20]:
# FIXME: This helper function should be rewritten using numpy's shift function. See below.
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [21]:
# Create test and training sets for one-step-ahead regression.
window_size = 2
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Original training data shape:")
print(train_X.shape)

# Reshape the input data into appropriate form for Keras.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New training data shape:")
print(train_X.shape)

Original training data shape:
(579, 2)
New training data shape:
(579, 1, 2)


In [22]:
trainer = pl.Trainer(
  max_epochs=100,
  gpus=0,
  weights_summary="top",
  gradient_clip_val=0.1,
  callbacks=[my_early_stop_callback],
  limit_train_batches=30,
)
net = NBeats.from_dataset(train_X, learning_rate=4e-3,
  log_interval=10, log_val_interval=1, weight_decay=1e-2,
  widths=[32, 512], backcast_loss_ratio=1.0)

trainer.fit(net, train_dataloader=train_dataloader,
  val_dataloaders=val_dataloader,
)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


AttributeError: 'numpy.ndarray' object has no attribute 'max_prediction_length'

In [ ]:
#Ouput predicted data to csv
#out = .astype(np.int)
#pd.DataFrame(out).to_csv(r'C:\Users\gurpr\Videos\BetaSci\NBeats_pred.csv', index=False, header=False)